<a href="https://colab.research.google.com/github/2kristint/scifinder/blob/main/scifinderAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chemicals pubchempy

In [ ]:
import json
import requests
from pprint import pprint
from IPython.display import SVG, HTML
import chemicals
import pandas as pd
from PIL import Image
import base64
from IPython.display import display
from bs4 import BeautifulSoup
import pubchempy as pcp
# import cirpy
# import io
detail_base_url = "https://commonchemistry.cas.org/api/detail?"

# Below needs to be turned into input

In [ ]:
chemical_list = 'ammonium hydroxide, Camphor-10-sulfonyl chloride, methylene chloride, magnesium sulfate, toluene, peracetic acid, sodium sulfate, hexane, 153221-24-0'.split(', ')

# I want the table to be the output

In [ ]:

def get_pug_sds(cas_number):
  data = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cas_number}/xml?heading=Safety-and-Hazards")

  html = BeautifulSoup(data.content, "xml")
  melting_points = html.find_all('String')
  mpset = []
  for melting_point in melting_points:
      mp = str(melting_point)
      mp = mp.replace('<String>','').replace('</String>','')
      mpset.append(mp)
  return str(mpset[2:5])

def scifinder(chemical_list):
  casrn = [chemicals.search_chemical(chemical).CASs for chemical in chemical_list]
  images = []
  mp = []
  name = []
  mol_wt = []
  density = []
  sds_list = []
  dataset = pd.DataFrame()
  for casrn1 in casrn:
    try:
      casrn1_data = requests.get(detail_base_url + "cas_rn=" + casrn1).json()
      # get svg image text
      images.append(casrn1_data["image"])
      # Get Boiling Point property
      mp.append(casrn1_data["experimentalProperties"][0]["property"][0:10])
      name.append(casrn1_data["name"])
      mol_wt.append(casrn1_data["molecularMass"])
      density.append(str(casrn1_data['experimentalProperties'][2])[33:45])

    except Exception as e:
      print(e)
      continue
  for casrn1 in chemical_list:
    try:
      # print(casrn1)
      sds_list.append(get_pug_sds(int(str(pcp.get_compounds(casrn1, 'name')[0]).replace('Compound(','').replace(')','  '))))
    except:
      continue

  dataset['name'] = pd.DataFrame(name)
  dataset['mol_wt'] = pd.DataFrame(mol_wt)
  dataset['density'] = pd.DataFrame(density)
  dataset['Melting Point'] = pd.DataFrame(mp)
  dataset['Hazards'] = pd.DataFrame(sds_list)
  display(dataset)
scifinder(chemical_list)

ValueError: A valid CAS number (153221-24-0) was recognized, but is not in the database

# I can't get the images to go into the table

In [ ]:
import requests
from pprint import pprint
import IPython.display as display

# Function to fetch chemical data from Common Chemistry API using CAS RN
def fetch_chemical_data(cas_rn):
    detail_base_url = "https://commonchemistry.cas.org/api/detail?"
    response = requests.get(detail_base_url, params={"cas_rn": cas_rn})
    if response.status_code == 200:
        return response.json()
    else:
        return None

casrn_list = [chemicals.search_chemical(chemical).CASs for chemical in chemical_list]

# Iterate through the list and fetch data for each CAS RN
for cas_rn in casrn_list:
  try:
    chemical_data = fetch_chemical_data(cas_rn)
    if chemical_data:
        if 'image' in chemical_data:
            display.display(display.SVG(chemical_data['image']))
    else:
        print("Failed to fetch data for CAS RN", cas_rn)
  except Exception as e:
    print(e)
    continue

ValueError: A valid CAS number (153221-24-0) was recognized, but is not in the database

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_pug_density(cas_number):
  data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/2244/xml?heading=Density")
  html = BeautifulSoup(data.content, "xml")
  densities = html.find_all('String')
  dset = []
  for density in densities:
    d = str(density)
    d = d.replace('<String>','').replace('</String>','')
    if 'g/cm' in d:
      return d

def get_pug_mol_wt(cas_number):
  data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/2244/xml?heading=Molecular+Weight")
  html = BeautifulSoup(data.content, "xml")
  densities = html.find_all('String')
  dset = []
  for density in densities:
    d = str(density)
    d = d.replace('<String>','').replace('</String>','')
    return d


def get_pug_mp(cas_number):
  data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/2244/xml?heading=Melting+Point")
  html = BeautifulSoup(data.content, "xml")
  melting_points = html.find_all('String')
  mpset = []
  for melting_point in melting_points:
      mp = str(melting_point)
      mp = mp.replace('<String>','').replace('</String>','')
      if 'C' in mp:
        mpset.append(mp)
  return(mpset[1])

def get_pug_sds(cas_number):
  data = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cas_number}/xml?heading=Safety-and-Hazards")

  html = BeautifulSoup(data.content, "xml")
  melting_points = html.find_all('String')
  mpset = []
  for melting_point in melting_points:
      mp = str(melting_point)
      mp = mp.replace('<String>','').replace('</String>','')
      mpset.append(mp)

  return str(mpset[2:5])


In [ ]:
def scifinder(chemical_list):
  casrn = [chemicals.search_chemical(chemical).CASs for chemical in chemical_list]
  images = []
  mp = []
  name = []
  mol_wt = []
  density = []
  sds_list = []
  dataset = pd.DataFrame()
  for casrn1 in casrn:
    try:
      # Get Boiling Point property
      mp.append(get_pug_mp(casrn1))
      mol_wt.append(get_pug_mp(casrn1))
      density.append(get_pug_density(casrn1))
    except Exception as e:
      print(e)
      continue
  for casrn1 in chemical_list:
      name.append(casrn1)
      sds_list.append(get_pug_sds(int(str(pcp.get_compounds(casrn1, 'name')[0]).replace('Compound(','').replace(')','  '))))

  dataset['name'] = pd.DataFrame(name)
  dataset['mol_wt'] = pd.DataFrame(mol_wt)
  dataset['density'] = pd.DataFrame(density)
  dataset['Melting Point'] = pd.DataFrame(mp)
  dataset['Hazards'] = pd.DataFrame(sds_list)
  display(dataset)
scifinder(chemical_list)

ValueError: A valid CAS number (153221-24-0) was recognized, but is not in the database

YOOOOO here's ur table
If it says the server is busy just reload again;
run your first 3 blocks of code: pip install, import statements, and chemical_list before running this

In [ ]:
#create list of cid numbers from names list
def create_cidList(nameList):
  compounds = []
  #create list of compounds from names
  for element in nameList:
    compound = pcp.get_compounds(element, 'name')
    compounds.append(compound[0])

  cids = []
  for ele in compounds:
    cids.append(ele.cid)

  return cids

#get images
#found this webpage: https://pubchem.ncbi.nlm.nih.gov/docs/imaging-services
def display_image(cidNum):
  # Fetch image from URL
  url = "https://pubchem.ncbi.nlm.nih.gov/image/imgsrv.fcgi?cid=" + str(cidNum)
  return '<img src="'+ url + '" style=max-height:124px;"/>'

#get melting point
def get_pug_melting_point(cidNum):
  data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/" + str(cidNum) + "/xml?heading=Melting+Point")
  html = BeautifulSoup(data.content, "xml")
  melting_points = html.find_all('String')
  for melting_point in melting_points:
      mp = str(melting_point)
      mp = mp.replace('<String>','').replace('</String>','')
      if 'C' in mp:
        return mp

#get density
def get_pug_density(cidNum):
  data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/" + str(cidNum) + "/xml?heading=Density")
  html = BeautifulSoup(data.content, "xml")
  densities = html.find_all('String')
  for density in densities:
      d = str(density)
      d = d.replace('<String>','').replace('</String>','')
      if 'C' in d:
        return d

#get sdf info
def get_pug_hazards(cidNum):
  data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/" + str(cidNum) + "/xml?heading=Hazards+Summary")
  html = BeautifulSoup(data.content, "xml")
  hazards = html.find_all('String')
  for hazard in hazards:
      h = str(hazard)
      h = h.replace('<String>','').replace('</String>','')
      if 'C' in h:
        return h

#creating table
def create_table(nameList):
  images = []
  mp = []
  name = []
  mol_wt = []
  density = []
  sds_list = []
  dataset = pd.DataFrame()

  compounds = []
  #create list of compounds from names
  for element in nameList:
    compound = pcp.get_compounds(element, 'name')
    compounds.append(compound[0])

  cids = create_cidList(nameList)

  #images
  for cidNum in cids:
    images.append(display_image(cidNum))

  # melting point
  for cidNum in cids:
    mp.append(get_pug_melting_point(cidNum))

  # density
  for cidNum in cids:
    density.append(get_pug_density(cidNum))

  #name
  for compoundName in nameList:
    name.append(compoundName)

  #molecular weight
  for compound in compounds:
    mol_wt.append(compound.molecular_weight)

  # sdf
  for cidNum in cids:
    sds_list.append(get_pug_hazards(cidNum))

  dataset['2D Models'] = pd.DataFrame(images)
  dataset['Name'] = pd.DataFrame(name)
  dataset['Molecular Weight'] = pd.DataFrame(mol_wt)
  dataset['Density'] = pd.DataFrame(density)
  dataset['Melting Point'] = pd.DataFrame(mp)
  dataset['Hazards'] = pd.DataFrame(sds_list)
  dataset = dataset.to_html(escape=False)
  display(HTML(dataset))

create_table(chemical_list)



,2D Models,Name,Molecular Weight,Density,Melting Point,Hazards
0,,ammonium hydroxide,35.046,About 0.90 @ 25 °C/25 °C,-58 °C (25%),"Corrosive to skin; [Quick CPC] High inhalation exposure can cause pulmonary edema. [HSDB] A corrosive substance that can cause injury to the skin, eyes and respiratory tract; Inhalation of high concentrations may cause laryngeal edema, respiratory tract inflammation, and pneumonia; Prolonged or repeated exposure to vapor or aerosol may cause injury to lungs; [ICSC] Solution of <28% aqueous ammonia: Causes burns; Short-term exposure causes smarting of the skin and first-degree burns; Second-degree burns can result from extended exposure; [CHRIS] Human inhalation of 408 ppm causes focal fibrosis (pneumoconiosis) and acute pulmonary edema; [RTECS] Causes burns; A lachrymator; Toxic by ingestion; Inhalation may cause corrosive injuries to upper respiratory tract and lungs; [Aldrich MSDS] See Ammonia."
1,,Camphor-10-sulfonyl chloride,250.74,None,None,None
2,,methylene chloride,84.93,"1.322 at 68 °F (USCG, 1999) - Denser than water; will sink",-95 °C,"Methylene chloride is predominantly used as a solvent. The acute (short-term) effects of methylene chloride inhalation in humans consist mainly of nervous system effects including decreased visual, auditory, and motor functions, but these effects are reversible once exposure ceases. The effects of chronic (long-term) exposure to methylene chloride suggest that the central nervous system (CNS) is a potential target in humans and animals. Human data are inconclusive regarding methylene chloride and cancer. Animal studies have shown increases in liver and lung cancer and benign mammary gland tumors following the inhalation of methylene chloride."
3,,magnesium sulfate,120.37,"Efflorescent crystals or powder; bitter, saline, cooling taste; density: 1.67; pH 6-7; soluble in water (g/100 ml): 71 @ 20 °C, 91 @ 40 °C; slightly soluble in alcohol; its aqueous soln is neutral; it loses 4 H2O @ 70-80 °C, 5 H2O @ 100 °C, 6 H2O @ 120 °C; loses last molecule of H2O @ about 250 °C; rapidly reabsorbing water when exposed to moist air; on exposure to dry air at ordinary temperatures it losses approx one H2O /Heptahydrate/",Decomposes @ 1124 °C,No listed effects of short-term or long-term exposure; [ICSC]
4,,toluene,92.14,"0.867 at 68 °F (USCG, 1999) - Less dense than water; will float",-94.9 °C,"Toluene is added to gasoline, used to produce benzene, and used as a solvent. Exposure to toluene may occur from breathing ambient or indoor air affected by such sources. The central nervous system (CNS) is the primary target organ for toluene toxicity in both humans and animals for acute (short-term) and chronic (long-term) exposures. CNS dysfunction and narcosis have been frequently observed in humans acutely exposed to elevated airborne levels of toluene; symptoms include fatigue, sleepiness, headaches, and nausea. CNS depression has been reported to occur in chronic abusers exposed to high levels of toluene. Chronic inhalation exposure of humans to toluene also causes irritation of the upper respiratory tract and eyes, sore throat, dizziness, and headache. Human studies have reported developmental effects, such as CNS dysfunction, attention deficits, and minor craniofacial and limb anomalies, in the children of pregnant women exposed to high levels of toluene or mixed solvents by inhalation. EPA has concluded that that there is inadequate information to assess the carcinogenic potential of toluene."
5,,peracetic acid,76.05,1.226 g/cu cm at 15 °C,-0.2 °C,"Highly corrosive to skin; [Quick CPC] Commercial solution is peracetic acid, hydrogen peroxide, acetic acid, and water (at equilibrium); [Merck Index] May decompose violently from shock, friction, or concussion; A strong oxidizing agent that reacts violently with combustible substances; Corrosive to skin, eyes, and respiratory tract; Inhalation may cause pulmonary edema; [ICSC] Does not exist in pure (100%) form; Commercially ava

# Here's your table

In [ ]:
#create list of cid numbers from names list
def create_cidList(nameList):
  compounds = []
  #create list of compounds from names
  for element in nameList:
    compound = pcp.get_compounds(element, 'name')
    compounds.append(compound[0])

  cids = []
  for ele in compounds:
    cids.append(ele.cid)

  return cids

#get images
#found this webpage: https://pubchem.ncbi.nlm.nih.gov/docs/imaging-services
def display_image(cidNum):
  # Fetch image from URL
  url = "https://pubchem.ncbi.nlm.nih.gov/image/imgsrv.fcgi?cid=" + str(cidNum)
  return '<img src="'+ url + '" style=max-height:124px;"/>'

#get melting point
def get_pug_melting_point(cidNum):
  data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/" + str(cidNum) + "/xml?heading=Melting+Point")
  html = BeautifulSoup(data.content, "xml")
  melting_points = html.find_all('String')
  for melting_point in melting_points:
      mp = str(melting_point)
      mp = mp.replace('<String>','').replace('</String>','')
      if 'C' in mp:
        return mp

#get density
def get_pug_density(cidNum):
  data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/" + str(cidNum) + "/xml?heading=Density")
  html = BeautifulSoup(data.content, "xml")
  densities = html.find_all('String')
  for density in densities:
      d = str(density)
      d = d.replace('<String>','').replace('</String>','')
      if 'C' in d:
        return d

#get sdf info
def get_pug_hazards(cidNum):
  data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/" + str(cidNum) + "/xml?heading=Hazards+Summary")
  html = BeautifulSoup(data.content, "xml")
  hazards = html.find_all('String')
  for hazard in hazards:
      h = str(hazard)
      h = h.replace('<String>','').replace('</String>','')
      if 'C' in h:
        return h

#creating table
def create_table(nameList):
  images = []
  mp = []
  name = []
  mol_wt = []
  density = []
  sds_list = []
  dataset = pd.DataFrame()

  compounds = []
  #create list of compounds from names
  for element in nameList:
    compound = pcp.get_compounds(element, 'name')
    compounds.append(compound[0])

  cids = create_cidList(nameList)

  #images
  for cidNum in cids:
    images.append(display_image(cidNum))

  # melting point
  for cidNum in cids:
    mp.append(get_pug_melting_point(cidNum))

  # density
  for cidNum in cids:
    density.append(get_pug_density(cidNum))

  #name
  for compoundName in nameList:
    name.append(compoundName)

  #molecular weight
  for compound in compounds:
    mol_wt.append(compound.molecular_weight)

  # sdf
  for cidNum in cids:
    sds_list.append(get_pug_hazards(cidNum))

  dataset['2D Models'] = pd.DataFrame(images)
  dataset['Name'] = pd.DataFrame(name)
  dataset['Molecular Weight'] = pd.DataFrame(mol_wt)
  dataset['Density'] = pd.DataFrame(density)
  dataset['Melting Point'] = pd.DataFrame(mp)
  dataset['Hazards'] = pd.DataFrame(sds_list)
  dataset = dataset.to_html(escape=False)
  display(HTML(dataset))

create_table(chemical_list)



,2D Models,Name,Molecular Weight,Density,Melting Point,Hazards
0,,ammonium hydroxide,35.046,About 0.90 @ 25 °C/25 °C,-58 °C (25%),"Corrosive to skin; [Quick CPC] High inhalation exposure can cause pulmonary edema. [HSDB] A corrosive substance that can cause injury to the skin, eyes and respiratory tract; Inhalation of high concentrations may cause laryngeal edema, respiratory tract inflammation, and pneumonia; Prolonged or repeated exposure to vapor or aerosol may cause injury to lungs; [ICSC] Solution of <28% aqueous ammonia: Causes burns; Short-term exposure causes smarting of the skin and first-degree burns; Second-degree burns can result from extended exposure; [CHRIS] Human inhalation of 408 ppm causes focal fibrosis (pneumoconiosis) and acute pulmonary edema; [RTECS] Causes burns; A lachrymator; Toxic by ingestion; Inhalation may cause corrosive injuries to upper respiratory tract and lungs; [Aldrich MSDS] See Ammonia."
1,,Camphor-10-sulfonyl chloride,250.74,None,None,None
2,,methylene chloride,84.93,"1.322 at 68 °F (USCG, 1999) - Denser than water; will sink",-95 °C,"Methylene chloride is predominantly used as a solvent. The acute (short-term) effects of methylene chloride inhalation in humans consist mainly of nervous system effects including decreased visual, auditory, and motor functions, but these effects are reversible once exposure ceases. The effects of chronic (long-term) exposure to methylene chloride suggest that the central nervous system (CNS) is a potential target in humans and animals. Human data are inconclusive regarding methylene chloride and cancer. Animal studies have shown increases in liver and lung cancer and benign mammary gland tumors following the inhalation of methylene chloride."
3,,magnesium sulfate,120.37,"Efflorescent crystals or powder; bitter, saline, cooling taste; density: 1.67; pH 6-7; soluble in water (g/100 ml): 71 @ 20 °C, 91 @ 40 °C; slightly soluble in alcohol; its aqueous soln is neutral; it loses 4 H2O @ 70-80 °C, 5 H2O @ 100 °C, 6 H2O @ 120 °C; loses last molecule of H2O @ about 250 °C; rapidly reabsorbing water when exposed to moist air; on exposure to dry air at ordinary temperatures it losses approx one H2O /Heptahydrate/",Decomposes @ 1124 °C,No listed effects of short-term or long-term exposure; [ICSC]
4,,toluene,92.14,"0.867 at 68 °F (USCG, 1999) - Less dense than water; will float",-94.9 °C,"Toluene is added to gasoline, used to produce benzene, and used as a solvent. Exposure to toluene may occur from breathing ambient or indoor air affected by such sources. The central nervous system (CNS) is the primary target organ for toluene toxicity in both humans and animals for acute (short-term) and chronic (long-term) exposures. CNS dysfunction and narcosis have been frequently observed in humans acutely exposed to elevated airborne levels of toluene; symptoms include fatigue, sleepiness, headaches, and nausea. CNS depression has been reported to occur in chronic abusers exposed to high levels of toluene. Chronic inhalation exposure of humans to toluene also causes irritation of the upper respiratory tract and eyes, sore throat, dizziness, and headache. Human studies have reported developmental effects, such as CNS dysfunction, attention deficits, and minor craniofacial and limb anomalies, in the children of pregnant women exposed to high levels of toluene or mixed solvents by inhalation. EPA has concluded that that there is inadequate information to assess the carcinogenic potential of toluene."
5,,peracetic acid,76.05,1.226 g/cu cm at 15 °C,-0.2 °C,"Highly corrosive to skin; [Quick CPC] Commercial solution is peracetic acid, hydrogen peroxide, acetic acid, and water (at equilibrium); [Merck Index] May decompose violently from shock, friction, or concussion; A strong oxidizing agent that reacts violently with combustible substances; Corrosive to skin, eyes, and respiratory tract; Inhalation may cause pulmonary edema; [ICSC] Does not exist in pure (100%) form; Commercially ava